# 节点高程和边等级

需要调用Google Maps Elevation API密钥 https://developers.google.com/maps/documentation/elevation/start

考虑您的API使用限制。 OSMnx将坐标四舍五入到小数点后五位（约1米）以批量容纳350个位置。
请注意，鉴于Google数据集的分辨率，存在一些空间误差。例如，在旧金山（分辨率为19米）中，丘陵公园中的几个边缘的坡度为50％以上，因为Google为其节点之一分配了与街道相邻的丘陵的海拔。

In [3]:
# from keys import google_elevation_api_key #replace this with your own API key!
import networkx as nx
import numpy as np
import osmnx as ox
ox.config(log_console=True, use_cache=True)
ox.__version__

'0.11dev'

In [ ]:
# get the street network for Fuyang
place = 'Langfang'
place_query = 'Langfang,Hebei,CN'
G = ox.graph_from_place(place_query,which_result=2,network_type='drive')

In [ ]:
# add elevation to each of the nodes, using the google elevation API, then calculate edge grades
G = ox.add_node_elevations(G, api_key=google_elevation_api_key)
G = ox.add_edge_grades(G)

## 计算一些统计指标

使用网络的无向表示，这样我们就不会过多计算双向街道（因为它们在每个方向上都有相反的边缘）。我们使用边缘坡度的绝对值，因为我们对陡度感兴趣，而不是方向性。

In [ ]:
edge_grades = [data['grade_abs'] for u, v, k, data in ox.get_undirected(G).edges(keys=True, data=True)]

In [ ]:
avg_grade = np.mean(edge_grades)
print('Average street grade in {} is {:.1f}%'.format(place, avg_grade*100))

med_grade = np.median(edge_grades)
print('Median street grade in {} is {:.1f}%'.format(place, med_grade*100))

# 按高程绘制节点

绘制从低（紫）到高（黄）的颜色。

In [ ]:
# get one color for each node, by elevation, then plot the network
nc = ox.get_node_colors_by_attr(G_proj, 'elevation', cmap='plasma', num_bins=20)
fig, ax = ox.plot_graph(G_proj, fig_height=6, node_color=nc, node_size=12, node_zorder=2, edge_color='#dddddd')

按等级绘制边

In [ ]:
# get a color for each edge, by grade, then plot the network
ec = ox.get_edge_colors_by_attr(G_proj, 'grade_abs', cmap='plasma', num_bins=100)
fig, ax = ox.plot_graph(G_proj, fig_height=6, edge_color=ec, edge_linewidth=0.8, node_size=0)

# 计算最短路径，考虑坡度阻抗

In [ ]:
# select an origin and destination node and a bounding box around them
origin = ox.get_nearest_node(G, (37.77, -122.426))
destination = ox.get_nearest_node(G, (37.773, -122.441))
bbox = ox.bbox_from_point((37.772, -122.434), distance=1500, project_utm=True)

In [ ]:
# define some edge impedance function here
def impedance(length, grade):
    penalty = grade ** 2
    return length * penalty

# add impedance and elevation rise values to each edge in the projected graph
# use absolute value of grade in impedance function if you want to avoid uphill and downhill
for u, v, k, data in G_proj.edges(keys=True, data=True):
    data['impedance'] = impedance(data['length'], data['grade_abs'])
    data['rise'] = data['length'] * data['grade']

首先找到最短路径，以最小化行程距离：

In [ ]:
route_by_length = nx.shortest_path(G_proj, source=origin, target=destination, weight='length')
fig, ax = ox.plot_graph_route(G_proj, route_by_length, bbox=bbox, node_size=0)

现在找到通过最小化阻抗（长度和坡度的函数）来避免倾斜的最短路径：

In [ ]:
route_by_impedance = nx.shortest_path(G_proj, source=origin, target=destination, weight='impedance')
fig, ax = ox.plot_graph_route(G_proj, route_by_impedance, bbox=bbox, node_size=0)

打印有关这两条路线的一些摘要统计信息：

In [ ]:
def print_route_stats(route):
    route_grades = ox.get_route_edge_attributes(G_proj, route, 'grade_abs')
    msg = 'The average grade is {:.1f}% and the max is {:.1f}%'
    print(msg.format(np.mean(route_grades)*100, np.max(route_grades)*100))

    route_rises = ox.get_route_edge_attributes(G_proj, route, 'rise')
    ascent = np.sum([rise for rise in route_rises if rise >= 0])
    descent = np.sum([rise for rise in route_rises if rise < 0])
    msg = 'Total elevation change is {:.0f} meters: a {:.0f} meter ascent and a {:.0f} meter descent'
    print(msg.format(np.sum(route_rises), ascent, abs(descent)))

    route_lengths = ox.get_route_edge_attributes(G_proj, route, 'length')
    print('Total trip distance: {:,.0f} meters'.format(np.sum(route_lengths)))

In [ ]:
# stats of route minimizing length
print_route_stats(route_by_length)

In [ ]:
# stats of route minimizing impedance (function of length and grade)
print_route_stats(route_by_impedance)